## Exercise 1

Poisson regression is a Generalized Linear Model, used to model count data. It takes the form

$$\mathbb{E}(\mu|x)=\exp(w_1\,x_1+\ldots+w_k\,x_k+b),$$

where the observed counts $y$ are drawn from a Poisson distribution on the expected counts: 

$$y_i \sim \text{Poisson}(\mu_i).$$

1. Download and import Load the smoking dataset from: [https://data.princeton.edu/wws509/datasets/#smoking](https://data.princeton.edu/wws509/datasets/#smoking). Then perform a train-test split on the data;


In [36]:
import numpy as np
import pandas as pd
import pyro
import torch
from sklearn.model_selection import train_test_split
import seaborn as sns

In [24]:
data = pd.read_csv("https://data.princeton.edu/wws509/datasets/smoking.raw",sep="\t",header=None)
data.columns = ["Age_Group","Smoking_Status","Population","Deaths"]
data.head()

,Age_Group,Smoking_Status,Population,Deaths
0,1,1,656,18
1,2,1,359,22
2,3,1,249,19
3,4,1,632,55
4,5,1,1067,117


* age at the start of follow-up: in five-year age groups coded 1 to 9 for 40-44, 45-49, 50-54, 55-59, 60-64, 65-69, 70-74, 75-79, 80+.
* smoking status: coded 1 = never smoked, 2 = smoked cigars or pipe only, 3 = smoked cigarettes and cigar or pipe, and 4 = smoked cigarettes only,
* population: number of male pensioners followed, and
* deaths: number of deaths in a six-year period.

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Age_Group       36 non-null     int64
 1   Smoking_Status  36 non-null     int64
 2   Population      36 non-null     int64
 3   Deaths          36 non-null     int64
dtypes: int64(4)
memory usage: 1.2 KB


Now we perform a train-test split on the data:
- **train data** - 80% of the observations will be used to perfom inference on our model
- **test data** - the remaining 20% will be used for testing the correctness of posterior predictions 

In [49]:
deaths = torch.tensor(data["Deaths"].values, dtype=torch.int)
predictors = torch.stack([torch.tensor(data[column].values,dtype=torch.int) 
                            for column in data.columns if column != "Deaths"],1)

X_train, X_test, y_train, y_test = train_test_split(predictors, deaths, test_size=0.20, 
                                                    random_state=42,shuffle=True)

print("X_train.shape =", X_train.shape,"\ny_train.shape =", y_train.shape)
print("\nX_test.shape =", X_test.shape,"\ny_test.shape =", y_test.shape)

X_train.shape = torch.Size([28, 3]) 
y_train.shape = torch.Size([28])

X_test.shape = torch.Size([8, 3]) 
y_test.shape = torch.Size([8])


2. Fit a Poisson bayesian regression model using the number of deaths as the response variable and the other columns as the explanatory variables;

**Need to study poisson regression before setting up the model, especially estimmation lambda^hat maybe I can check the SMDS notes**

The target variable in our model is the number of deaths and we wish to infer the parameters corresponding to the following predictors

$$
\text{Deaths}=\text{exp}(w_0\cdot\text{Age_Group}+w_1\cdot\text{Smoking_Status}+w_2\cdot\text{Population}+b) + \epsilon
$$

We set a normal prior on $w$, a Log-Normal on the bias term $b$ and a uniformly distributed std for the gaussian noise on $\hat{y}$

\begin{align*}
w&\sim\mathcal{N}(0,1)\\
b&\sim\text{LogNormal}(0,1)\\
\hat{\mu_i}&= w x + b\\
y &\sim \text{Poisson}(\mu_i).
\end{align*}

Then we define the family of posterior distributions, by setting a Gamma distribution on $w$ and a Log-Normal on $b$, and run SVI inference on $(x,y)$ data.

Notice the prior distribution on the bias term makes this regression problem analytically intractable.


3. Evaluate the regression fit on test data using MAE and MSE error metrics.

## Exercise 2

The Iris dataset contains petal and sepal length and width for three different types of Iris flowers: Setosa, Versicolour, and Virginica.

1. Import the Iris dataset from `sklearn`:
```
from sklearn import datasets
iris = datasets.load_iris()
```
and perform a train-test split on the data.

2. Fit a multinomial bayesian logistic regression model on the four predictors petal length/width and sepal length/width. 

3. Evaluate your bayesian classifier on test data: compute the overall test accuracy and class-wise accuracy for the three different flower categories.